### 加载一些需要用到的Python Modules。

In [1]:
import mdtraj
import nglview
import tempfile

from openbabel import pybel

_ColormakerRegistry()

In [2]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

sc = SparkContext("local", "DisplaySparkVinaResults")

### 建立一个SparkSqlContext

注意一下`read.parquet(...)`里面要加一个文件夹。这个文件夹里面是我们SparkVina输出的结果。
如果你一直用我给的路径，就应该是这个路径。
如果不是，你可以返回到Jupyter主页，右上角点New，New一个terminal。通过`ls`来找一下这个folder。

这个folder里面只有类似这种文件：
+ _SUCCESS
+ ....parquet
+ ....parquet

In [3]:
sqlContext = SQLContext(sc)

df = sqlContext.read.parquet('./output_test_2');

The following is the corresponding Parquet message type:

```
message spark_schema {
  required binary name (UTF8);
  required int32 num_models;
  required double affinity_mean;
  required double affinity_std;
  required group vina_results (LIST) {
    repeated group list {
      required group element {
        required double affinity;
        required binary ligand_string (UTF8);
      }
    }
  }
}
```

+ name: The name of the small compound
+ num_models: The number of models obtained from the docking.
+ affinity_mean: The average score of docking this small compound to the protein.
+ affinity_std: The standard deviation of docking this small compound to the protein.
+ vina_results: The details of the result.

In [4]:
df.select('name', 'num_models', 'affinity_mean', 'affinity_std').show()

+----------------+----------+-------------------+-------------------+
|            name|num_models|      affinity_mean|       affinity_std|
+----------------+----------+-------------------+-------------------+
|ZINC000044171185|         8| -5.566175985538466|0.16081887160117989|
|ZINC000009255826|         8|-5.5394295579982815|0.19226259989484354|
+----------------+----------+-------------------+-------------------+



### 提取详细结果

In [5]:
vina_results = df.select('vina_results').collect()

### 查看vina_results的内容（可参看以上的schema）

In [6]:
vina_results

[Row(vina_results=[Row(random_seed=-1944351971, vina_result=[Row(affinity=-5.74657186271299, docked_pdbqt='\nREMARK  Name = ZINC000044171185\nREMARK                            x       y       z     vdW  Elec       q    Type\nREMARK                         _______ _______ _______ _____ _____    ______ ____\nROOT\nATOM      1  C   LIG    1      171.546-109.786-112.188  0.00  0.00    +0.180 C \nATOM      2  O   LIG    1      171.303-110.972-111.400  0.00  0.00    -0.340 OA\nATOM      3  C   LIG    1      169.867-111.074-111.298  0.00  0.00    +0.430 C \nATOM      4  C   LIG    1      169.383-109.623-111.081  0.00  0.00    +0.180 C \nATOM      5  C   LIG    1      170.512-108.755-111.683  0.00  0.00    +0.190 C \nENDROOT\nBRANCH   3   6\nATOM      6  N   LIG    1      169.494-111.914-110.156  0.00  0.00    -0.490 N \nATOM      7  C   LIG    1      168.227-112.295-109.814  0.00  0.00    +0.380 A \nATOM      8  N   LIG    1      168.258-113.038-108.748  0.00  0.00    -0.440 NA\nATOM      9  

### 提取第一个小分子（数据库中第一行）所有model中最好的model

In [7]:
# 0的意思为数据库第一行
unwrapped_results = vina_results[0]['vina_results']

lowest_affinity = 0.0
best_docked_model = None
for row in unwrapped_results:
    for result_row in row.vina_result:
        if result_row.affinity < lowest_affinity:
            lowest_affinity = result_row.affinity
            best_docked_model = result_row

### 查看最好的model

In [8]:
best_docked_model

Row(affinity=-5.74657186271299, docked_pdbqt='\nREMARK  Name = ZINC000044171185\nREMARK                            x       y       z     vdW  Elec       q    Type\nREMARK                         _______ _______ _______ _____ _____    ______ ____\nROOT\nATOM      1  C   LIG    1      171.546-109.786-112.188  0.00  0.00    +0.180 C \nATOM      2  O   LIG    1      171.303-110.972-111.400  0.00  0.00    -0.340 OA\nATOM      3  C   LIG    1      169.867-111.074-111.298  0.00  0.00    +0.430 C \nATOM      4  C   LIG    1      169.383-109.623-111.081  0.00  0.00    +0.180 C \nATOM      5  C   LIG    1      170.512-108.755-111.683  0.00  0.00    +0.190 C \nENDROOT\nBRANCH   3   6\nATOM      6  N   LIG    1      169.494-111.914-110.156  0.00  0.00    -0.490 N \nATOM      7  C   LIG    1      168.227-112.295-109.814  0.00  0.00    +0.380 A \nATOM      8  N   LIG    1      168.258-113.038-108.748  0.00  0.00    -0.440 NA\nATOM      9  C   LIG    1      169.545-113.181-108.337  0.00  0.00    -0.2

### 可视化最好的model

In [9]:
# 一个把上面显示的这个PDBQT的字符串加载成MDTraj的结构模型
def load_pdbqt_string(pdbqt_string):
    pdb_model = None
    tmp = tempfile.NamedTemporaryFile()
    mol = pybel.readstring('pdbqt', pdbqt_string)
    
    pdbfile = tmp.name + '.pdb'
    mol.write('pdb', pdbfile)
    return mdtraj.load_pdb(pdbfile)

In [10]:
pdb_model = load_pdbqt_string(best_docked_model.docked_pdbqt)

### 查看一下加载的这个小分子

+ 有多少个原子？
+ 有多少化学键？

In [11]:
pdb_model.topology

<mdtraj.Topology with 1 chains, 1 residues, 31 atoms, 33 bonds at 0x7f6749fd3f90>

### 利用NGLWidget可视化小分子和蛋白结合

In [12]:
view = nglview.NGLWidget()
view.add_pdbid('4zph')
view.add_trajectory(pdb_model)

In [13]:
view

NGLWidget()

### ZINC数据库中这个小分子的信息

In [17]:
# Index 0就是第一个小分子在数据库中的索引号

url = 'https://zinc15.docking.org/substances/{}/'.format(df.select('name').collect()[0].name)
url

'https://zinc15.docking.org/substances/ZINC000044171185/'

In [18]:
import IPython

IPython.display.IFrame(src=url, width=1400, height=1000)

### 结论

1. 从上图看，小分子确实被dock到了晶体结构我们想dock的地方。
2. 并且它和晶体结构中的抑制剂是重合的，意思也就是说，被安放到了正确的（我们期待的地方）。

### 扩展

1. 可以查看一下其他affinity差一些的model，看看他们被安放到了什么位置，是什么构象？